###  free variables (aka nonlocal variables)

In [1]:
from typing import Callable  # type hinting


def inspect_vars(o: Callable) -> None:
    for i in ("co_varnames", "co_freevars"):
        print(i, eval("o.__code__." + i))

In [2]:
Average = float  # type aliasing: meaning float and Average will be treated as interchangeable synonyms


def maker() -> Callable[[float], Average]: #returns a function Callable[[float], Avearge]: func(float) -> Average
    so_far = []#it's not a global variable: it looks like it but it is also local! What is it? NON LOCAL variable

    def avg(new_value: float) -> Average:
        so_far.append(new_value)
        return sum(so_far) / len(so_far)

    return avg


av = maker()# it creates a box and attach to it the label av and then returns me that box
print(av(9))
print(av(11))
print(av(40))

9.0
10.0
20.0


In [3]:
inspect_vars(av)

co_varnames ('new_value',)
co_freevars ('so_far',)


In [4]:
def maker_buggy() -> Callable[[float], Average]:
    _sum = 0
    _count = 0

    def avg(new_value: float) -> Average:
        #PROBLEMS: local variable '_sum' referenced before assignment!!!!
        _sum += new_value # --> _sum = _sum + new_value SEE IT AS LOCAL!
        _count += 1  # --> _count = _count + 1
        #Why this didn't happen with a list? so_far = []? Because we used .append()
        #Python every time you use += put it as a local variable (+ always returns a new object)!
        return _sum / _count

    return avg


av = maker_buggy()
av(9)

UnboundLocalError: local variable '_sum' referenced before assignment

In [5]:
inspect_vars(av)

co_varnames ('new_value', '_sum', '_count')
co_freevars ()


In [6]:
def maker_right() -> Callable[[float], Average]:
    _sum = 0
    _count = 0

    def avg(new_value: float) -> Average:
        nonlocal _sum, _count  # <----
        _sum += new_value
        _count += 1
        return _sum / _count

    return avg


av = maker_right()
print(av(9))
print(av(11))
print(av(40))

9.0
10.0
20.0


In [7]:
inspect_vars(av)

co_varnames ('new_value',)
co_freevars ('_count', '_sum')


### closure
A closure is a function that retains the bindings (mantiene il legame) of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the **defining scope** is no longer available.